In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_system_data(system, window=100, figsize=(10, 8)):
    """
    Plot and save time series data with moving averages for a given system.
    
    :param system: String identifier for the system (used for file naming)
    :param window: Size of the moving average window
    :param figsize: Size of the figure (width, height)
    """
    def plot_with_moving_average(ax, df, x_column, y_column):
        """
        Plot original data and its moving average on a given axis.
        """
        ax.plot(df[x_column], df[y_column], label='Original', alpha=0.5)
        rolling_mean = df[y_column].rolling(window=window, center=True, min_periods=1).mean()
        ax.plot(df[x_column], rolling_mean, label=f'Moving Average (window={window})', color='red')
        ax.set_xlabel(x_column)
        ax.set_ylabel(y_column)
        ax.set_title(f'{y_column} Over Time')
        ax.legend()

    # Read the CSV file
    file_path = f"output/{system}.out"
    try:
        df = pd.read_csv(file_path, sep=',')
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return

    # Get column names for indices 3, 4, 5, 6
    columns_to_plot = df.columns[3:7]

    # Create a single figure with four subfigures
    fig, axs = plt.subplots(2, 2, figsize=figsize)
    fig.suptitle(f'System: {system}', fontsize=16)

    # Flatten the 2x2 array of axes for easier iteration
    axs = axs.flatten()

    # Plot for each specified column
    for i, column in enumerate(columns_to_plot):
        plot_with_moving_average(axs[i], df, 'Time (ps)', column)

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)  # Adjust to prevent title overlap

    # Save the figure
    output_dir = "output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_file = os.path.join(output_dir, f"{system}_plot.png")
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()  # Close the figure to free up memory

    print(f"Plot saved as {output_file}")

In [24]:
plot_system_data("S1_Best_A")
plot_system_data("S1_Best_AB")
plot_system_data("S2_Best_A")
plot_system_data("S2_Best_AB")

Plot saved as output/S1_Best_A_plot.png
Error: File output/S1_Best_AB.out not found.
Plot saved as output/S2_Best_A_plot.png
Plot saved as output/S2_Best_AB_plot.png


In [2]:
import os
import imageio
from PIL import Image

def create_gif_from_pngs(input_folder, output_gif, duration=0.5, loop=0, compress_factor=0.5):
    """
    Create a compressed GIF from all PNG files in a folder.
    
    :param input_folder: Path to the folder containing PNG files
    :param output_gif: Path and filename for the output GIF
    :param duration: Duration of each frame in seconds (default: 0.5)
    :param loop: Number of times to loop the GIF. 0 means loop forever (default: 0)
    :param compress_factor: Factor to resize images for compression (default: 0.5)
    """
    # Get all PNG files in the input folder
    png_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.png')]
    png_files.sort()  # Sort files to ensure correct order

    images = []
    for png_file in png_files:
        file_path = os.path.join(input_folder, png_file)
        # Open the image and resize it for compression
        with Image.open(file_path) as img:
            # Calculate new size
            new_size = (int(img.width * compress_factor), int(img.height * compress_factor))
            # Resize the image
            img_resized = img.resize(new_size, Image.LANCZOS)
            # Convert to RGB mode if it's not already
            if img_resized.mode != 'RGB':
                img_resized = img_resized.convert('RGB')
            # Append to the list
            images.append(img_resized)

    # Save as GIF
    imageio.mimsave(output_gif, images, duration=duration, loop=loop)
    print(f"GIF created and saved as {output_gif}")

# Example usage
input_folder = '/Users/jakublala/Downloads/output 6/movie'
output_gif = '/Users/jakublala/Downloads/output 6/animated.gif'
create_gif_from_pngs(input_folder, output_gif)

GIF created and saved as /Users/jakublala/Downloads/output 6/animated.gif
